<a href="https://colab.research.google.com/github/EmanoneL/data_analysis_pet_project/blob/main/desk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Импорт библиотек

In [ ]:
import pandas as pd # Для работы с DataFrame
import numpy as np # Для математических операций с DataFrame
import matplotlib.pyplot as plt # Для работы с графиками и гистограммами
import scipy.stats as stats # Некоторые функции для работы со статистикой
import seaborn as sns # Для ящика с усами
from scipy.stats import kstest # Критерий Колмогорова-Смирнова
from scipy.stats import chi2_contingency # Критерий хи-квадрат

### Пользовательские функции

In [ ]:
# Формирует новый столбец(фичу) в DataFrame по данным из столбца price_doc
def price_doc_to_class(df):
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    X['class'] = ''

    X.loc[(y <= 3000000), 'class'] = 'cheap'
    X.loc[(y > 3000000) & (y <= 11000000), 'class'] = 'normal'
    X.loc[y > 11000000, 'class'] = 'expensive'

    X = pd.concat([X, y], axis=1)
    return X

In [ ]:
# Проверяет близость к нормальному распределению для фичи по критерию Колмогорова-Смирнова
def checkStandart(df, feature_name):
  data = (df[feature_name] - df[feature_name].mean()) / df[feature_name].std()

  # Выполняем K-S тест для проверки нормальности
  stat, p_value = kstest(data, 'norm')
  print(f'Statistic: {stat}, p-value: {p_value}')

  # Интерпретация результата
  alpha = 0.05
  if p_value > alpha:
      print('Распределение близко к нормальному (не отклоняем H0)')
  else:
      print('Распределение отличается от нормального (отклоняем H0)')

In [ ]:
def breushPaganTest(X, y):
  X_const = sm.add_constant(X)
  model_sm = sm.OLS(y, X_const).fit()
  test_stat, p_value, _, _ = het_breuschpagan(model_sm.resid, model_sm.model.exog)

  print(f'Test Statistic: {test_stat}')
  print(f'p-value: {p_value}')

  if p_value < 0.05:
      print("Отвергаем нулевую гипотезу: наличие гетероскедастичности.")
  else:
      print("Не отвергаем нулевую гипотезу: гомоскедастичность сохраняется.")

### Загрузка данных

In [ ]:
df = pd.read_csv('dataset.csv', index_col=0) # Загружаем файл в DataFrame
df.head(10) # Выводим первые 10 результатов

FileNotFoundError: [Errno 2] No such file or directory: 'dataset.csv'

In [ ]:
df.info()

Заметим сразу, что в выборке нет пропусков

**Описание датасета**

* **full_sq** - общая площадь
* **build_year** - год постройки
* **price_doc** - цена квартиры

#Дескриптивный анализ

In [ ]:
df.hist(bins=50)

In [ ]:
df['price_doc'].describe()

In [ ]:
df['full_sq'].describe()

In [ ]:
sns.boxplot(x=df['price_doc'])

In [ ]:
sns.boxplot(x=df['full_sq'])

In [ ]:
df = df[
    (df['price_doc'] <= df['price_doc'].quantile(0.95))
    & (df['full_sq'] <= df['full_sq'].quantile(0.95))
]

In [ ]:
df.hist(bins=50)

In [ ]:
df.describe()

In [ ]:
checkStandart(df, 'full_sq')

In [ ]:
checkStandart(df, 'price_doc')

In [ ]:
from scipy.stats import spearmanr

# Применям ранговую корреляцию Спирмана, так как распределение не нормальное
corr, p_value = spearmanr(df['price_doc'], df['full_sq'])
print(f"Spearman's correlation: {corr}, p-value: {p_value}")

# Интерпретация
alpha = 0.05
if p_value < alpha:
    print('Корреляция значима (отклоняем H0)')
else:
    print('Корреляция незначима (не отклоняем H0)')

In [ ]:
df2 = price_doc_to_class(df) # Добавляем категориальную переменную

In [ ]:
# Строим таблицу сопряженности
contingency_table = pd.crosstab(df2['build_type'], df2['class'])
print(contingency_table)

In [ ]:
# Критерий хи-квадрат для проверки статистической гипотезы о независимости этих переменных
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2}")
print(f"p-value: {p_value}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:\n", expected)

# Интерпретация результатов
alpha = 0.05
if p_value < alpha:
    print("Отвергаем нулевую гипотезу - существует связь между переменными.")
else:
    print("Не отвергаем нулевую гипотезу - связи между переменными не обнаружено.")

## Регрессионный анализ


У нас есть корреляция между full_sq, build_type и price_doc
Так как у нас есть категориальный признак, преобразуем его в несколько числовых с помощью метода one-hot


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

In [ ]:
# Определение признаков и целевой переменной
X = df[['full_sq', 'build_type']]
x = df['full_sq']
y = df['price_doc']

In [ ]:
X = pd.get_dummies(df, 'build_type',dtype=int)
X

In [ ]:
# Разделение данных (например, 80% обучающая, 20% тестовая)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Добавляем константу для intercept
x = sm.add_constant(x)

# Создаем и обучаем модель OLS
model = sm.OLS(y, x)
results = model.fit()

# Предсказанные значения и остатки
predicted_values = results.fittedvalues
residuals = results.resid

# Строим график остатков
plt.scatter(predicted_values, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Предсказанные значения')
plt.ylabel('Остатки')
plt.title('График остатков')
plt.show()

In [ ]:
# Гистограмма стандартизированных признаков
# Рассчитываем стандартное отклонение остатков
std_residuals = np.std(residuals)

# Стандартизированные остатки
standardized_residuals = residuals / std_residuals

# Построение гистограммы стандартизированных остатков
plt.figure(figsize=(10, 6))
sns.histplot(standardized_residuals, kde=True, bins=30, color='skyblue')
plt.xlabel('Стандартизированные остатки')
plt.ylabel('Частота')
plt.title('Гистограмма стандартизированных остатков')
plt.axvline(0, color='red', linestyle='--')  # линия на уровне 0
plt.show()

In [ ]:
results.summary()

p-значение меньше, чем выбранный уровень значимости, это говорит о том, что коэффициенты являются значимыми

Если p-значение (Prob F-statistic) меньше стандартного уровня значимости, это означает, что хотя бы одна из независимых переменных оказывает статистически значимое влияние на зависимую переменную.
В данном случае p-значение чрезвычайно мало (меньше 0.05), что подтверждает, что модель регрессии в целом является статистически значимой

**Нормальность остатков**:

Omnibus и Jarque-Bera (JB) — это тесты, которые проверяют гипотезу о нормальности распределения остатков.
Omnibus: значение 17.270 с p-значением 0.000.
Jarque-Bera (JB): значение 20.465 с p-значением 3.60e-05.
Оба теста указывают на отклонение остатков от нормального распределения (p < 0.05), что может свидетельствовать о нарушении допущения нормальности остатков.

**Автокорреляция остатков**:

Durbin-Watson (DW) тест используется для проверки автокорреляции остатков. Значение DW равно 1.972, что близко к 2, и это указывает на отсутствие значительной автокорреляции (при 2 автокорреляция отсутствует).

**Гетероскедастичность**:

ниже проведен тест Бреуша-Пагана, который показал, что дисперсия остатков не является постоянной, следовательно, присутсвует гетероскедастичность




In [ ]:
intercept, slope = results.params
print(f"Уравнение регрессии: y = {intercept} + {slope} * X")

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor


# Функция для вычисления VIF
def calculate_vif(dataframe):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = dataframe.columns
    vif_data["VIF"] = [variance_inflation_factor(dataframe.values, i) for i in range(dataframe.shape[1])]
    return vif_data


vif_result = calculate_vif(x)

print(vif_result)


Нет признаков избыточности переменных среди независимых переменных, так как VIF для full_sq находится на приемлемом уровне.


